## Test notebook
Here we tested on sf-xs-test and tokyo-xs from the precious traine model


In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('..') # append parent directory, we need it

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
# import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from utils.validation import get_validation_recalls
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
os.environ["OMP_NUM_THREADS"] = "1"

In [ ]:
MEAN=[0.485, 0.456, 0.406]; STD=[0.229, 0.224, 0.225]

IM_SIZE = (320, 320)

def input_transform(image_size=IM_SIZE):
    return T.Compose([
        # T.Resize(image_size, interpolation=T.InterpolationMode.BICUBIC),
		T.Resize(image_size,  interpolation=T.InterpolationMode.BILINEAR),
        
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])

In this project, we provide for each benchmark (or test dataset) a Dataset Class that encapsulates images sequentially as follows: 

$[R_1, R_2, ..., R_n, Q_1, Q_2, ..., Q_m]$ where $R_i$ are the reference images and $Q_i$ are the queries. We keep the number of references and queries as variables in the object so that we can split into references/queries later when evaluating. We also store a ground_truth matrix that indicates which references are prositives for each query.

**Note:** make sure that for every [BenchmarkClass].py, the global variable DATASET_ROOT (where each dataset images are located) is well initialized, otherwise you won't be able to run the following steps. Also, GT_ROOT is the location of the precomputed ground_truth and filenames that WE PROVIDED (by default in ../datasets/).

In [ ]:
from dataloaders.val.SFDataset import SFDataset
from dataloaders.val.TokyoDataset import TokyoDataset


def get_val_dataset(dataset_name, input_transform=input_transform()):
    dataset_name = dataset_name.lower()
    
    if 'sfxs_val' in dataset_name:
        ds = SFDataset(input_transform = input_transform, val_test='val')
    elif 'sfxs_test' in dataset_name:
        ds = SFDataset(input_transform = input_transform, val_test='test')
    elif 'tokyo' in dataset_name:
        ds = TokyoDataset(input_transform = input_transform)
    else:
        raise ValueError
    
    num_references = ds.num_references
    num_queries = ds.num_queries
    ground_truth = ds.ground_truth
    return ds, num_references, num_queries, ground_truth

We define a function to which we give a model, a dataloader and it returns the resulting representations

In [ ]:
def get_descriptors(model, dataloader, device):
    descriptors = []
    with torch.no_grad():
        for batch in tqdm(dataloader, 'Calculating descritptors...'):
            imgs, labels = batch
            output = model(imgs.to(device)).cpu()
            descriptors.append(output)

    return torch.cat(descriptors)

## Let's now load a pre-trained model and evaluating on all benchmarks


In [ ]:
from main import VPRModel
from pathlib import Path

# define which device you'd like run experiments on (cuda:0 if you only have one gpu)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = VPRModel(backbone_arch='resnet18', 
                 layers_to_crop=[4],
                 agg_arch='gem',
                 agg_config={'p': 3},
        )

file = Path('outputs\checkpoints\HT_gem3_adamw_lr1e-3_wd1\resnet18[gem3_AdamW_lr1e-3_wd1]sf_xs_val_epoch(14)_step(29310)_R1[0.5886]_R5[0.7370].ckpt')
state_dict = torch.load(file) # link to the trained weights
# model.load_state_dict(state_dict, strict=False)
model.load_state_dict(state_dict['state_dict'])
model.eval()
model = model.to(device)

val_dataset_names = ['sfxs_val', 'sfxs_test', 'tokyo']
batch_size = 40

for val_name in val_dataset_names:
    val_dataset, num_references, num_queries, ground_truth = get_val_dataset(val_name)
    val_loader = DataLoader(val_dataset, num_workers=4, batch_size=batch_size)
    print(f'Evaluating on {val_name}')
    descriptors = get_descriptors(model, val_loader, device)
    
    print(f'Descriptor dimension {descriptors.shape[1]}')
    r_list = descriptors[ : num_references]
    q_list = descriptors[num_references : ]

    recalls_dict, preds = get_validation_recalls(r_list=r_list,
                                                q_list=q_list,
                                                k_values=[1, 5, 10, 15, 20, 25],
                                                gt=ground_truth,
                                                print_results=True,
                                                dataset_name=val_name,
                                                faiss_gpu=False
                                                )
    del descriptors
    print('========> DONE!\n\n')


resnet18sf_xs_val_epoch(04)_step(9770)_R1[0.5307]_R5[0.6902].ckpt
Evaluating on sfxs_val


Calculating descritptors...:   0%|          | 0/401 [00:00<?, ?it/s]

Descriptor dimension 256


+----------------------------------------------------------+
|                 Performance on sfxs_val                  |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 53.06 | 69.06 | 75.43 | 78.87 | 81.32 | 83.02 |
+----------+-------+-------+-------+-------+-------+-------+
========> DONE!


Evaluating on sfxs_test


Calculating descritptors...:   0%|          | 0/705 [00:00<?, ?it/s]

Descriptor dimension 256


+----------------------------------------------------------+
|                 Performance on sfxs_test                 |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 21.30 | 33.00 | 40.70 | 46.20 | 49.20 | 51.00 |
+----------+-------+-------+-------+-------+-------+-------+
========> DONE!


Evaluating on tokyo


...

Descriptor dimension 256


+----------------------------------------------------------+
|                 Performance on sfxs_val                  |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 57.12 | 72.68 | 78.22 | 81.33 | 83.71 | 85.40 |
+----------+-------+-------+-------+-------+-------+-------+
========> DONE!


Evaluating on sfxs_test


Calculating descritptors...:   0%|          | 0/705 [00:00<?, ?it/s]

Descriptor dimension 256


+----------------------------------------------------------+
|                 Performance on sfxs_test                 |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 23.00 | 37.10 | 43.80 | 48.60 | 51.70 | 54.20 |
+----------+-------+-------+-------+-------+-------+-------+
========> DONE!


Evaluating on tokyo


Calculating descritptors...:   0%|          | 0/328 [00:00<?, ?it/s]

Descriptor dimension 256


+----------------------------------------------------------+
|                   Performance on tokyo                   |
+----------+-------+-------+-------+-------+-------+-------+
|    K     |   1   |   5   |   10  |   15  |   20  |   25  |
+----------+-------+-------+-------+-------+-------+-------+
| Recall@K | 34.29 | 51.75 | 59.37 | 64.13 | 69.52 | 73.33 |
+----------+-------+-------+-------+-------+-------+-------+
========> DONE!


